# 観光スポットをたくさん取得してくる

In [1]:
from urllib.parse import parse_qsl
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from requests import get as GET
import re
import numpy as np

In [2]:
# キーワードを使って検索する
keywords = ['東京|大阪|名古屋', '観光']
top_num = 20

In [3]:
html = GET('https://www.google.co.jp/search?num=' +
           str(top_num+1)+'&q=' + ' '.join(keywords)).text
soup = BeautifulSoup(html, 'lxml')

urls = []
for el in soup.select("h3.r a"):
    title = el.get_text()
    match = re.search("検索結果", title)
    if match:
        continue
    urls.append(dict(parse_qsl(urlparse(el.get("href")).query))["q"])
    print(title)
    print("  ", urls[-1])

名古屋観光するなら！地元民が教える定番・人気スポット29選 | LINE ...
   https://www.travel.co.jp/guide/matome/99/
東京観光はコレ! 専門家おすすめ 外国人にも人気なスポット30選 | LINE ...
   https://www.travel.co.jp/guide/matome/93/
東京観光おすすめスポット65選！名所 穴場 最新スポット完全網羅 | 楽天 ...
   https://travel.rakuten.co.jp/mytrip/ranking/spot-tokyo/
地元民が大阪旅行におすすめする必見の観光・グルメ ... - TABI CHANNEL
   https://tabichannel.com/article/107/osaka_kanko
東京観光おすすめレジャー50選！絶対行きたい観光地の決定版はコレ
   https://travel.mynavi.jp/kanko/article/2578/
2018年 名古屋市で絶対外さない観光スポット 10選 【定番から穴場まで ...
   https://www.tripadvisor.jp/Attractions-g298106-Activities-Nagoya_Aichi_Prefecture_Tokai_Chubu.html
東京の観光スポットランキングTOP10 - じゃらんnet
   https://www.jalan.net/kankou/130000/
名古屋の観光スポットランキングTOP10 - じゃらんnet
   https://www.jalan.net/kankou/230000/230200/
2018年 名古屋で絶対外せない！おすすめ観光スポット＆ランキング ...
   https://gurutabi.gnavi.co.jp/p23/n2301/
定番から穴場まで！絶対外せない大阪のおすすめ人気観光スポットTOP40
   https://retrip.jp/articles/1849/
観光・遊ぶ | 名古屋観光情報公式サイト 名古屋コンシェルジュ
   https://www.nagoya-info.jp/view/
【2018年11月版】意外な穴場もあり？大阪でおすすめの観光スポット 

In [4]:
# それぞれのページから観光情報っぽい部分を取得
def get_sightseeing(url):
    r = GET(url)
    soup = BeautifulSoup(r.content, 'html.parser')  # 文字化け対策:chardet導入済み
    title = [text.string for text in soup.find_all("title")]
#     h1 = [text.string for text in soup.find_all("h1")]
    h2 = [text.string for text in soup.find_all("h2")]
    h3 = [text.string for text in soup.find_all("h3")]
    h4 = [text.string for text in soup.find_all("h4")]
    tags = [h2, h3, h4]

    # 観光サイトはあるタグに見出しをたくさん置くため，数の多いタグを採用
    index = np.argmax([len(h2), len(h3), len(h4)])
    for i in range(100):
        try:
            tags[index].remove(None)  # Noneがあれば，削除しておく
        except ValueError:
            break
    return title[0], tags[index]

In [5]:
titles = []
tags = []
for url in urls:
    title, tag = get_sightseeing(url)
    titles.append(title)
    tags.append(tag)
    print("title:", title)
    for text in tag:
        print("・・・", text)

title: 名古屋観光するなら！地元民が教える定番・人気スポット29選 | LINEトラベルjp 旅行ガイド
・・・ 尾張名古屋は城で持つ！名古屋観光の定番「名古屋城」
・・・ 時空を越えて愛される回遊式日本庭園「徳川園」
・・・ 名古屋城から徳川園にいたる「文化のみち」を散策
・・・ 創建当初からの文化財は必見！尾張徳川家の菩提寺｢建中寺｣
・・・ 名古屋観光の朝はやっぱり喫茶店で「モーニング」
・・・ 有名作品のロケ地としておなじみ「名古屋市市政資料館」
・・・ “モノづくり”の楽しさを再認識「トヨタ産業技術記念館」
・・・ 都心部一等地の民間中央市場「マルナカ食品センター」
・・・ 陶磁器の歴史と技術と魅力を学べる「ノリタケの森」
・・・ 名古屋観光の拠点に！極上ステイが楽しめる「ホテル」
・・・ 絶品麺グルメ「きしめん・味噌煮込みうどん・台湾ラーメン」
・・・ 名古屋栄のシンボル「名古屋テレビ塔」は日本最古の電波塔
・・・ 屋上観覧車にビアガーデン「三越百貨店」が面白い
・・・ まるで宇宙船！？ビル街の真ん中で空中散歩「オアシス２１」
・・・ ヤミツキ手羽先対決「風来坊　VS　世界の山ちゃん」
・・・ 世界最大プラネタリウムだけじゃない「名古屋市科学館」
・・・ 大須観音の門前町「大須商店街」をぶらぶら街歩き
・・・ まだまだあります！個性豊かな「名古屋めし」
・・・ コアラブームの火付け役「東山動植物園」で一日遊ぼう
・・・ 街なかに佇む「パワースポット」を巡って運気をアップ
・・・ スイーツ？ごはん？進化を続ける「名古屋グルメ」
・・・ 織田信長も必勝祈願「熱田神宮」で三種の神器のパワーを！
・・・ 一杯で四度楽しめる本場の「ひつまぶし」
・・・ 有名デザイナーが手掛ける庭園「名古屋港ブルーボネット」
・・・ 名古屋のシンボル”シャチ”に会える「名古屋港水族館」
・・・ 大人も子供も楽しめる体験型鉄道博物館「リニア・鉄道館」
・・・ 戦いの舞台「桶狭間」と東海道の宿場町「有松」
・・・ 名古屋コーチンのひよこは必見「農業センターdelaふぁーむ」
・・・ ハズレなし！地元民も愛する定番の「おみやげ」
・・・ 名古屋観光は歴史もグルメも科学の最先端も
・・・ あわせて読みたい！関連旅行ガイド
・・・ 旅行プランをさがそう！
・・・ この記事の目次
tit

title: 【2018年】自分らしく東京を旅する♪観光ポイント＆エリアの特徴を紹介！ | icotto（イコット）
・・・ 東京都の観光ポイント♪
・・・ 東京・丸の内エリアをぶらぶら観光♪おすすめスポット6選
・・・ 歴史を感じる銀座・丸の内エリアの美術館＆周辺スポットをご紹介！
・・・ 【2018年】初心者でも安心♪浅草観光のポイント＆王道観光スポットはココ！
・・・ 【保存版】巨大なレジャースポット「東京スカイツリー」観光
・・・ 上野に観光ならココ！上野を堪能できるおすすめスポット10選
・・・ 上野アメ横を満喫したいならこのスポットがおすすめ！6選
・・・ これぞ秋葉原！個性的な客室やプランがあるホテル7選
・・・ 初心者向けの店もあるよ♡オタクの聖地・秋葉原のメイドカフェ＆バー6選
・・・ 大人の東京観光は「銀座」へ♪銀ぶらにおすすめのスポット7選
・・・ 銀座デートはどこに行く？2人で行きたいおしゃれデートスポット12選
・・・ 雰囲気の良い写真が撮れる！お台場のおすすめ撮影スポット10選
・・・ 無料バス「東京ベイシャトル」でめぐるお台場エリア観光スポット&グルメ15選
・・・ 四季折々の自然が都会で楽しめる！「新宿御苑」にいこう♪
・・・ ディープな街、新宿「思い出横丁」でハズせないグルメ
・・・ 【渋谷】甘くてジューシーなフルーツたっぷり♡絶品「スイーツカフェ」6選
・・・ おしゃれエリア【奥渋谷】にある個性派グルメ＆ショップ10選
・・・ かわいいがいっぱい！原宿・竹下通りのショップ＆スポット7選
・・・ 表参道のオシャレカフェでランチを楽しむ♪おすすめ店10選
・・・ 東京散歩しましょ♪恵比寿・代官山エリアのおすすめスポット7選
・・・ 1日満喫できる本屋！代官山「蔦屋書店」と「T-SITE」の魅力♪
・・・ サンシャイン60の展望台が凄い！体験展望台「スカイサーカス」に行こう！
・・・ 池袋にある「天空のオアシス」！サンシャイン水族館に行ってみよう！
・・・ 東京の夜に酔いしれよう。六本木エリアのオシャレなホテル7選
・・・ 六本木周辺のレストランならココ！世界中のグルメを味わえるおすすめの10店
・・・ 家族連れやデートでも楽しめる！上野動物園のパンダに会いに行こう♪
・・・ 古きよき日本を感じるなら日本橋へ。1日で巡る観光スポット8選